In [1]:
print("Something's wrong")

Something's wrong


In [1]:
import pydytuesday

pydytuesday.get_date('2025-08-05')

Trying to fetch README from: https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2025/2025-08-05/readme.md
Successfully fetched README from: https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2025/2025-08-05/readme.md
Successfully saved income_inequality_processed.csv to /home/serban/Documents/Training/nicegui/datasets/20250805/income_inequality_processed.csv
Successfully saved income_inequality_raw.csv to /home/serban/Documents/Training/nicegui/datasets/20250805/income_inequality_raw.csv
Successfully saved meta.yaml to /home/serban/Documents/Training/nicegui/datasets/20250805/meta.yaml
Successfully saved reduction-in-income-inequality.png to /home/serban/Documents/Training/nicegui/datasets/20250805/reduction-in-income-inequality.png


In [20]:
import polars as pl
# import pandas as pd

# raw_data: pl.DataFrame = pl.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-08-05/income_inequality_raw.csv")
raw_data = pl.read_csv("income_inequality_raw.csv", infer_schema_length=None, null_values='NA')


raw_data.describe()

statistic,Entity,Code,Year,gini_disposable__age_total,gini_market__age_total,population_historical,owid_region
str,str,str,f64,f64,f64,f64,str
"""count""","""58925""","""55521""",58925.0,616.0,596.0,58890.0,"""272"""
"""null_count""","""0""","""3404""",0.0,58309.0,58329.0,35.0,"""58653"""
"""mean""",null,null,1612.889741,0.318461,0.472504,5.0861e7,null
"""std""",null,null,1402.605752,0.060016,0.048747,3.0027e8,null
"""min""","""Afghanistan""","""ABW""",-10000.0,0.209,0.364,1.0,"""Africa"""
"""25%""",null,null,1833.0,0.278,0.436,148096.0,null
"""50%""",null,null,1902.0,0.312,0.481,1.410429e6,null
"""75%""",null,null,1967.0,0.346,0.51,6.773156e6,null
"""max""","""Zimbabwe""","""ZWE""",2023.0,0.626,0.72,8.0917e9,"""South America"""


In [27]:
data = raw_data.cast({"Year": pl.Int32})\
.rename(lambda column_name: column_name.lower())\
.rename({"gini_disposable__age_total": "gini_after_tax"})\
.rename({"gini_market__age_total": "gini_pre_tax"})\
.filter(pl.col("gini_after_tax").is_not_null() & pl.col("gini_pre_tax").is_not_null())

data.describe()

statistic,entity,code,year,gini_after_tax,gini_pre_tax,population_historical,owid_region
str,str,str,f64,f64,f64,f64,str
"""count""","""596""","""596""",596.0,596.0,596.0,596.0,"""2"""
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,"""594"""
"""mean""",null,null,2012.770134,0.317666,0.472504,3.5680e7,null
"""std""",null,null,8.018205,0.06075,0.048747,1.0182e8,null
"""min""","""Australia""","""AUS""",1976.0,0.209,0.364,292127.0,"""North America"""
"""25%""",null,null,2009.0,0.275,0.436,5.088409e6,null
"""50%""",null,null,2014.0,0.31,0.481,1.001661e7,null
"""75%""",null,null,2018.0,0.346,0.51,3.6353292e7,null
"""max""","""United States""","""ZAF""",2023.0,0.626,0.72,1.3603e9,"""North America"""


In [28]:
owid_regions = data.unique("code")
print(owid_regions)

shape: (45, 7)
┌─────────────┬──────┬──────┬────────────────┬──────────────┬───────────────────────┬─────────────┐
│ entity      ┆ code ┆ year ┆ gini_after_tax ┆ gini_pre_tax ┆ population_historical ┆ owid_region │
│ ---         ┆ ---  ┆ ---  ┆ ---            ┆ ---          ┆ ---                   ┆ ---         │
│ str         ┆ str  ┆ i32  ┆ f64            ┆ f64          ┆ i64                   ┆ str         │
╞═════════════╪══════╪══════╪════════════════╪══════════════╪═══════════════════════╪═════════════╡
│ Spain       ┆ ESP  ┆ 2007 ┆ 0.313          ┆ 0.449        ┆ 45516452              ┆ null        │
│ Israel      ┆ ISR  ┆ 2011 ┆ 0.371          ┆ 0.494        ┆ 7470971               ┆ null        │
│ Costa Rica  ┆ CRI  ┆ 2010 ┆ 0.472          ┆ 0.505        ┆ 4555924               ┆ null        │
│ Belgium     ┆ BEL  ┆ 2018 ┆ 0.258          ┆ 0.49         ┆ 11428564              ┆ null        │
│ Turkey      ┆ TUR  ┆ 2011 ┆ 0.403          ┆ 0.427        ┆ 74215158              ┆